In [2]:
%matplotlib qt
%load_ext autoreload
%autoreload 2

import sys; sys.path.insert(0, r'C:\Users\Lukas\Documents\projects\invert')
import mne
import pickle as pkl
from time import time
from scipy.spatial.distance import cdist
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


from invert.forward import get_info, create_forward_model
from invert.util import pos_from_forward

pp = dict(surface='white', hemi='both', verbose=0)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load Data

In [26]:
with open("forward_model/64ch_info.pkl", "rb") as f:
    info = pkl.load(f)
    
sim_type = "extended"
fn = f"evaluation/sim_and_preds_{sim_type}.pkl"
with open(fn, 'rb') as f:
    stc_dict_ext, x_test_ext, y_test_ext, sim_info_ext, _, _ = pkl.load(f)

sim_type = "single"
fn = f"evaluation/sim_and_preds_{sim_type}.pkl"
with open(fn, 'rb') as f:
    stc_dict_sing, x_test_sing, y_test_sing, sim_info_sing, _, _ = pkl.load(f)



sim_info = pd.concat([sim_info_sing, sim_info_ext])
sim_info["sample"] = sim_info.index.values
sim_info["sample"][500:] += 500

fwd = mne.read_forward_solution("forward_model/64ch_ico3-fwd.fif", verbose=0)
fwd = mne.convert_forward_solution(fwd, force_fixed=True)
pos = pos_from_forward(fwd)
source_model = fwd['src']
vertices = [source_model[0]['vertno'], source_model[1]['vertno']]
distances = cdist(pos, pos)
argsorted_distance_matrix = np.argsort(distances, axis=1)

sim_type = "single"
fn = f"results/results_{sim_type}.pkl"
with open(fn, 'rb') as f:
    results_single = pkl.load(f)

sim_type = "extended"
fn = f"results/results_{sim_type}.pkl"
with open(fn, 'rb') as f:
    results_extended = pkl.load(f)

df_single = pd.DataFrame(results_single)
df_single["Source Extend"] = "Single"
df_extended = pd.DataFrame(results_extended)
df_extended["Source Extend"] = "Extended"

df = pd.concat([df_single, df_extended])

n_methods = len(set(df.Method.values))
sample = np.tile(np.arange(sim_info.shape[0]/2).astype(int), n_methods*2)
df["Sample"] = sample
df["Sample"][df["Source Extend"]=="Extended"] += 500

snrs = []
n_sources = []
iscs = []
for i in range(df.shape[0]):
    sample = df["Sample"].values[i]
    snr = sim_info.loc[sim_info["sample"] == sample]["snr"].values[0]
    n_source = sim_info.loc[sim_info["sample"] == sample]["n_sources"].values[0]
    isc = sim_info.loc[sim_info["sample"] == sample]["inter_source_correlations"].values[0]
    snrs.append(snr)
    n_sources.append(n_source)
    iscs.append(isc)

df["SNR"] = snrs
df["Number of sources"] = n_sources
df["Inter Source Corr."] = iscs

snr_min = df["SNR"].min()
snr_max = df["SNR"].max()
print(snr_min, snr_max)


bins = [[0.1, 10], [10, 50], [50, 100]]
bins_idc = np.arange(len(bins))
bin_names = ["0.1 - 5", "5 - 20", "20 - 100"]
snr_bins_df = []
for i in range(df.shape[0]):
    snr = df["SNR"].values[i]
    for bin_idx, bin in zip(bins_idc, bins):
        if snr < bin[1] and snr >= bin[0]:
            snr_bins_df.append(bin_idx)
            break
df["SNR Bins"] = snr_bins_df

bins = [[0., 0.25], [0.25, 0.5], [0.5, 0.75], [0.75, 1.0]]
bins_idc_isc = np.arange(len(bins))
bin_names_isc = ["0.0 - 0.25", "0.25 - 0.5", "0.5 - 0.75", "0.75 - 1.0"]
snr_bins_df = []
for i in range(df.shape[0]):
    snr = df["Inter Source Corr."].values[i]
    for bin_idx, bin in zip(bin_names_isc, bins):
        if snr < bin[1] and snr >= bin[0]:
            snr_bins_df.append(bin_idx)
            break
df["Inter Source Corr. Bins"] = snr_bins_df

df.Method[df.Method=="Convexity Champagne"] = "Champagne"
df.Method[df.Method=="FLEX-MUSIC"] = "FLEX\nMUSIC"
df.Method[df.Method=="RAP-MUSIC"] = "RAP\nMUSIC"
df.Method[df.Method=="FLEX-AP"] = "FLEX\nAP"

df.Method[df.Method=="eLORETA"] = "eLOR"
df.rename(columns = {col: col.replace("-", " ").replace("_", " ") for col in df.columns}, inplace = True)
df

    No patch info available. The standard source space normals will be employed in the rotation to the local surface coordinates....
    Changing to fixed-orientation forward solution with surface-based source orientations...
    [done]


C:\Users\Lukas\AppData\Local\Temp\ipykernel_4460\1049479816.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sim_info["sample"][500:] += 500
C:\Users\Lukas\AppData\Local\Temp\ipykernel_4460\1049479816.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Sample"][df["Source Extend"]=="Extended"] += 500
C:\Users\Lukas\AppData\Local\Temp\ipykernel_4460\1049479816.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Sample"][df

0.15242376550852638 99.9391300720064


C:\Users\Lukas\AppData\Local\Temp\ipykernel_4460\1049479816.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Method[df.Method=="Convexity Champagne"] = "Champagne"
C:\Users\Lukas\AppData\Local\Temp\ipykernel_4460\1049479816.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Method[df.Method=="FLEX-MUSIC"] = "FLEX\nMUSIC"
C:\Users\Lukas\AppData\Local\Temp\ipykernel_4460\1049479816.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

,Mean Squared Error,Normalized Mean Squared Error,Mean Localization Error,AUC,Corr,EMD,Sparsity pred,Sparsity true,Active True,Active Pred,Method,Time Make,Time Apply,Source Extend,Sample,SNR,Number of sources,Inter Source Corr.,SNR Bins,Inter Source Corr. Bins
0,1.405776e-06,0.000727,1.808331,1.000000,0.839119,31803.370742,2.754643,2.735979,0.007009,0.007009,FLEX\nAP,4.965194,0.017000,Single,0,68.381468,9,0.668884,2,0.5 - 0.75
1,1.052850e-06,0.000548,2.650103,1.000000,0.856380,15604.332712,2.431206,2.502891,0.005452,0.005452,FLEX\nAP,4.044031,0.017004,Single,1,50.088634,7,0.864407,2,0.75 - 1.0
2,1.663345e-06,0.003760,8.739474,0.804500,0.124459,151962.470942,5.027023,2.685532,0.007788,0.040498,FLEX\nAP,4.220513,0.015999,Single,2,60.302221,10,0.566941,2,0.5 - 0.75
3,4.772751e-07,0.000324,0.000000,1.000000,0.891075,32114.047432,1.712634,1.658405,0.002336,0.002336,FLEX\nAP,1.147999,0.022003,Single,3,37.571678,3,0.793194,1,0.75 - 1.0
4,2.508332e-07,0.000306,0.000000,1.000000,0.903478,28941.188162,2.149745,2.148218,0.003894,0.003894,FLEX\nAP,1.676061,0.040000,Single,4,71.867545,5,0.729668,2,0.5 - 0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3495,2.419239e-09,0.045322,31.383571,0.505102,-0.029503,173464.990334,33.673920,2.132007,0.005452,1.000000,MCMV,1.577009,0.023767,Extended,995,82.069528,1,0.143505,2,0.0 - 0.25
3496,3.181122e-07,0.037776,16.077184,0.546088,-0.006785,183310.480477,31.798026,3.512715,0.016355,1.000000,MCMV,1.561979,0.028945,Extended,996,67.766453,3,0.060212,2,0.0 - 0.25
3497,1.441686e-05,0.113342,25.266254,0.549773,0.028640,176110.315752,31.857874,5.853608,0.052960,1.000000,MCMV,1.803436,0.041499,Extended,997,36.917728,5,0.946631,1,0.75 - 1.0
3498,8.505749e-07,0.036736,12.924884,0.572567,-0.051398,177579.780994,30.144243,4.907093,0.034268,1.000000,MCMV,1.577302,0.024806,Extended,998,49.663244,4,0.045051,1,0.0 - 0.25


# Box- and Scatter- Plots

In [27]:
sns.set(style="whitegrid", font_scale=1.)

tick_params = dict(
    axis='y',          # changes apply to the y-axis
    which='both',      # both major and minor ticks are affected
    left=True,      # ticks along the bottom edge are off
    right=False,         # ticks along the top edge are off
    labelbottom=False
)

medianprops = {
    "linewidth": 2,
    "linestyle": "dashed"
    }

order = ['FLEX\nAP', 'FLEX\nMUSIC', 'AP', 'RAP\nMUSIC', 'Champagne', 'MCMV', 'eLOR']

#================================================================================#

fig1 = plt.figure(figsize=(16,5))
plt.subplot(131)
sns.boxplot(data=df, x="Method", y="Mean Localization Error", hue="Source Extend", order=order, medianprops=medianprops)
plt.tick_params(**tick_params)
plt.ylim(-3, 43)
plt.ylabel("Mean Localization Error [mm]")
plt.gca().get_legend().remove()

plt.subplot(132)
g = sns.boxplot(data=df, x="Method", y="EMD", hue="Source Extend", order=order, medianprops=medianprops)
plt.tick_params(**tick_params)
plt.gca().get_legend().remove()

plt.subplot(133)
sns.boxplot(data=df, x="Method", y="Mean Squared Error", hue="Source Extend", order=order, medianprops=medianprops)
plt.ylim(-0.05e-5, 1e-5)
plt.tick_params(**tick_params)

plt.tight_layout(pad=2)
df.rename(columns={"Sparsity pred": "Sparsity"}, inplace=True)

#================================================================================#

fig2 = plt.figure(figsize=(10,5))
plt.subplot(121)
sns.boxplot(data=df, x="Method", y="Sparsity", hue="Source Extend", order=order, medianprops=medianprops)
plt.tick_params(**tick_params)
plt.gca().get_legend().remove()

# plt.subplot(122)
# g = sns.boxplot(data=df, x="Method", y="Time", hue="Source Extend", order=order, medianprops=medianprops)
# g.set_yscale("log")
# plt.ylabel("Computation Time [s]")
# plt.tick_params(**tick_params)

# plt.tight_layout(pad=2)

from scipy.stats import pearsonr
sns.set(style="white", font_scale=1.)
tick_params = dict(
    axis='both',          # changes apply to the y-axis
    which='both',      # both major and minor ticks are affected
    direction="inout",
    left=True,      # ticks along the bottom edge are off
    bottom=True,
    right=False,         # ticks along the top edge are off
    labelbottom=True
)

#================================================================================#

fig3 = plt.figure(figsize=(8, 6))
for i, method in enumerate(order):
    print(method)
    plt.subplot(3,3,i+1)
    df_temp = df[df.Method==method]

    a = df_temp["Sparsity true"].values
    b = df_temp["Sparsity"].values
    nans = (np.isnan(a) | np.isnan(b))
    a[nans] = np.nanmedian(a)
    b[nans] = np.nanmedian(b)
    r, p = pearsonr(a, b)
    # print(f"{method} sparsity: r = {r:.2f}")

    
    sns.scatterplot(data=df_temp, y="Sparsity", x="Sparsity true")
    plt.gca().set_aspect('equal', adjustable='box')

    

    maxval = np.max( [plt.ylim()[1], plt.xlim()[1]])
    plt.ylim(0,  maxval)
    plt.xlim(0, maxval)
    
    
    plt.plot([0, maxval], [0, maxval], 'k')
    plt.ylabel("Predicted Sparsity")
    plt.xlabel("True Sparsity")
    plt.title(method)
    r_text = f"r = {r:.2f}"
    if p<0.001:
        r_text += " ***"
    elif p<0.01:
        r_text += " **"
    elif p<0.05:
        r_text += " *"
    plt.text(maxval/2.6, maxval/1.2, r_text)
    xticks = plt.xticks()[0]
    yticks = plt.yticks()[0]
    longest_ticks = xticks if len(xticks)>len(yticks) else yticks
    plt.yticks(longest_ticks)
    plt.xticks(longest_ticks)
    plt.ylim(0,  maxval)
    plt.xlim(0, maxval)
    plt.tick_params(**tick_params)

    
plt.tight_layout(pad=2)

#================================================================================#

fig4 = plt.figure(figsize=(13,4))
plt.subplot(131)
sns.boxplot(data=df, hue="Method", y="Mean Localization Error", x="SNR Bins", hue_order=order)
plt.xticks(bins_idc, bin_names)
plt.xlabel("SNR")
plt.ylabel("Mean Localization Error [mm]")
plt.gca().get_legend().remove()

plt.subplot(132)
sns.boxplot(data=df, hue="Method", y="EMD", x="SNR Bins", hue_order=order)
plt.xticks(bins_idc, bin_names)
plt.xlabel("SNR")
plt.gca().get_legend().remove()

plt.subplot(133)
sns.boxplot(data=df, hue="Method", y="Mean Squared Error", x="SNR Bins", hue_order=order)
plt.xticks(bins_idc, bin_names)
plt.xlabel("SNR")
plt.ylim(-0.05e-5, 1e-5)
plt.legend(loc='upper center', bbox_to_anchor=(1.3, 1.0))

plt.tight_layout(pad=2)


fig5 = plt.figure(figsize=(13,4))
plt.subplot(131)
sns.pointplot(data=df, hue="Method", y="Mean Localization Error", x="Number of sources", hue_order=order, errorbar="se")
plt.gca().get_legend().remove()
plt.ylabel("Mean Localization Error [mm]")

plt.subplot(132)
sns.pointplot(data=df, hue="Method", y="EMD", x="Number of sources", hue_order=order, errorbar="se")
plt.gca().get_legend().remove()

plt.subplot(133)
sns.pointplot(data=df, hue="Method", y="Mean Squared Error", x="Number of sources", hue_order=order, errorbar="se")
plt.ylim(-0.05e-5, 3.19e-6)
plt.legend(loc='upper center', bbox_to_anchor=(1.3, 1.0))

plt.tight_layout(pad=2)


fig6 = plt.figure(figsize=(13,4))
plt.subplot(131)
sns.boxplot(data=df, hue="Method", y="Mean Localization Error", x="Inter Source Corr. Bins", hue_order=order)
plt.xticks(bins_idc_isc, bin_names_isc)
plt.xlabel("ISC")
plt.ylabel("Mean Localization Error [mm]")
plt.gca().get_legend().remove()

plt.subplot(132)
sns.boxplot(data=df, hue="Method", y="EMD", x="Inter Source Corr. Bins", hue_order=order)
plt.xticks(bins_idc_isc, bin_names_isc)
plt.xlabel("ISC")
plt.gca().get_legend().remove()

plt.subplot(133)
sns.boxplot(data=df, hue="Method", y="Mean Squared Error", x="Inter Source Corr. Bins", hue_order=order)
plt.xticks(bins_idc_isc, bin_names_isc)
plt.xlabel("ISC")
plt.ylim(-0.05e-5, 1e-5)
plt.legend(loc='upper center', bbox_to_anchor=(1.3, 1.0))

plt.tight_layout(pad=2)

# fig1.savefig("figures/accuracy.png", dpi=600)
# fig2.savefig("figures/sparsity_time.png", dpi=600)
# fig3.savefig("figures/extend_estimation.png", dpi=600)
# fig4.savefig("figures/snr.png", dpi=600)
# fig5.savefig("figures/source_number.png", dpi=600)
# fig6.savefig("figures/source_correlation.png", dpi=600)

FLEX
AP
FLEX
MUSIC
AP
RAP
MUSIC
Champagne
MCMV
eLOR


# Table: Metrics

In [51]:
df["Source Extend"]

0         Single
1         Single
2         Single
3         Single
4         Single
          ...   
3495    Extended
3496    Extended
3497    Extended
3498    Extended
3499    Extended
Name: Source Extend, Length: 7000, dtype: object

In [48]:
df_table = df.copy()

df_table.Method[df.Method=="FLEX\nMUSIC"] = "FLEX-MUSIC"
df_table.Method[df.Method=="RAP\nMUSIC"] = "RAP-MUSIC"
df_table.Method[df.Method=="FLEX\nAP"] = "FLEX-AP"
df_table = df_table.drop(columns=["Source Extend", "Inter Source Corr. Bins"])
df_table = df_table.groupby("Method").median()
# df_table = df_table[["Mean Localization Error", "EMD", "Mean Squared Error", "Sparsity", "Time"]]
df_table = df_table[["Mean Localization Error", "EMD", "Mean Squared Error", "Sparsity"]]
for col in df_table.columns:
    if not "Squared" in col:
        df_table[col] = df_table[col].round(2)

print(df_table.to_latex())

\begin{tabular}{lrrrr}
\toprule
 & Mean Localization Error & EMD & Mean Squared Error & Sparsity \\
Method &  &  &  &  \\
\midrule
AP & 15.830000 & 164990.500000 & 0.000001 & 2.230000 \\
Champagne & 8.470000 & 127440.180000 & 0.000001 & 6.300000 \\
FLEX-AP & 1.740000 & 63920.380000 & 0.000001 & 3.020000 \\
FLEX-MUSIC & 1.020000 & 63564.170000 & 0.000001 & 2.640000 \\
MCMV & 19.220000 & 186151.540000 & 0.000001 & 30.280000 \\
RAP-MUSIC & 9.810000 & 157840.060000 & 0.000001 & 1.900000 \\
eLOR & 18.670000 & 185825.320000 & 0.000001 & 28.340000 \\
\bottomrule
\end{tabular}



C:\Users\Lukas\AppData\Local\Temp\ipykernel_4460\2835578023.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table.Method[df.Method=="FLEX\nMUSIC"] = "FLEX-MUSIC"
C:\Users\Lukas\AppData\Local\Temp\ipykernel_4460\2835578023.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table.Method[df.Method=="RAP\nMUSIC"] = "RAP-MUSIC"
C:\Users\Lukas\AppData\Local\Temp\ipykernel_4460\2835578023.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

In [50]:
df_table.sort_values("Mean Localization Error")

,Mean Localization Error,EMD,Mean Squared Error,Sparsity
Method,,,,
FLEX-MUSIC,1.02,63564.17,5.945561e-07,2.64
FLEX-AP,1.74,63920.38,6.063368e-07,3.02
Champagne,8.47,127440.18,6.067987e-07,6.30
RAP-MUSIC,9.81,157840.06,7.922813e-07,1.90
AP,15.83,164990.50,1.088178e-06,2.23
eLOR,18.67,185825.32,1.057870e-06,28.34
MCMV,19.22,186151.54,1.348662e-06,30.28


# Statistics

In [54]:
import pingouin as pg
dvs = ["Mean Localization Error", "EMD", "Mean Squared Error", "Sparsity"]
df_single = df[df["Source Extend"]=="Single"]
df_ext = df[df["Source Extend"]=="Extended"]
for dv in dvs:
    aov = pg.anova(data=df, dv=dv, between=["Method", "Source Extend"], detailed=True)
    post_hoc_all = pg.pairwise_gameshowell(data=df, dv=dv, between="Method", effsize="cohen")
    post_hoc_single = pg.pairwise_gameshowell(data=df_single, dv=dv, between="Method", effsize="cohen")
    post_hoc_ext = pg.pairwise_gameshowell(data=df_ext, dv=dv, between="Method", effsize="cohen")
    
    print("###########################################")
    print(f"{dv} between Methods:")
    print("###########################################")
    display(aov)
    print("\n Post-hoc All Sources")
    display(post_hoc_all)
    print("\n Post-hoc Single Sources")
    display(post_hoc_single)
    print("\n Post-hoc Extended Sources")
    display(post_hoc_ext)
    
    print("\n")


###########################################
Mean Localization Error between Methods:
###########################################


,Source,SS,DF,MS,F,p-unc,np2
0,Method,154936.832339,6.0,25822.805390,475.954821,0.000000e+00,0.290727
1,Source Extend,47394.608324,1.0,47394.608324,873.556996,5.610896e-181,0.111415
2,Method * Source Extend,20880.070642,6.0,3480.011774,64.142077,7.630801e-78,0.052348
3,Residual,377992.778671,6967.0,54.254741,NaN,NaN,NaN



 Post-hoc All Sources


,A,B,mean(A),mean(B),diff,se,T,df,pval,cohen
0,AP,Champagne,14.513113,8.589458,5.923655,0.343477,17.246145,1905.778004,0.000000e+00,0.771548
1,AP,FLEX\nAP,14.513113,5.249083,9.264030,0.351637,26.345425,1945.379045,4.282130e-13,1.178594
2,AP,FLEX\nMUSIC,14.513113,5.859739,8.653374,0.386512,22.388384,1986.673361,0.000000e+00,1.003358
3,AP,MCMV,14.513113,16.573097,-2.059984,0.369522,-5.574722,1991.385585,5.884774e-07,-0.249378
4,AP,RAP\nMUSIC,14.513113,10.359313,4.153800,0.412840,10.061513,1935.368748,3.674838e-14,0.451671
5,AP,eLOR,14.513113,17.829086,-3.315973,0.333795,-9.934170,1841.602241,0.000000e+00,-0.444445
6,Champagne,FLEX\nAP,8.589458,5.249083,3.340375,0.312884,10.676075,1991.314902,5.895284e-14,0.477449
7,Champagne,FLEX\nMUSIC,8.589458,5.859739,2.729719,0.351624,7.763185,1867.100982,0.000000e+00,0.348098
8,Champagne,MCMV,8.589458,16.573097,-7.983639,0.332858,-23.985108,1943.403890,0.000000e+00,-1.072647
9,Champagne,RAP\nMUSIC,8.589458,10.359313,-1.769855,0.380375,-4.652922,1749.862271,7.192838e-05,-0.209076



 Post-hoc Single Sources


...

,A,B,mean(A),mean(B),diff,se,T,df,pval,cohen
0,AP,Champagne,2.159420,6.703096,-4.543676,0.095397,-47.628919,1084.371081,4.555245e-13,-2.130030
1,AP,FLEX\nAP,2.159420,3.149048,-0.989628,0.044365,-22.306434,1442.707895,0.000000e+00,-0.997574
2,AP,FLEX\nMUSIC,2.159420,2.770076,-0.610656,0.040316,-15.146691,1546.499750,3.329559e-13,-0.677381
3,AP,MCMV,2.159420,29.801361,-27.641941,0.082719,-334.168652,1114.002422,0.000000e+00,-14.944476
4,AP,RAP\nMUSIC,2.159420,1.843299,0.316121,0.026181,12.074619,1981.925856,0.000000e+00,0.539993
5,AP,eLOR,2.159420,28.241935,-26.082514,0.048709,-535.473169,1359.786407,0.000000e+00,-23.947088
6,Champagne,FLEX\nAP,6.703096,3.149048,3.554048,0.101596,34.982117,1352.294832,0.000000e+00,1.564448
7,Champagne,FLEX\nMUSIC,6.703096,2.770076,3.933020,0.099894,39.371748,1279.815371,3.728129e-13,1.760758
8,Champagne,MCMV,6.703096,29.801361,-23.098265,0.123272,-187.377130,1954.825770,0.000000e+00,-8.379760
9,Champagne,RAP\nMUSIC,6.703096,1.843299,4.859797,0.095073,51.116299,1070.304370,0.000000e+00,2.285990



 Post-hoc Single Sources


,A,B,mean(A),mean(B),diff,se,T,df,pval,cohen
0,AP,Champagne,2.168621,6.459155,-4.290534,0.124214,-34.541421,548.912014,5.073719e-14,-2.184591
1,AP,FLEX\nAP,2.168621,2.535125,-0.366504,0.052735,-6.949900,817.287441,1.571933e-10,-0.439550
2,AP,FLEX\nMUSIC,2.168621,2.179844,-0.011223,0.043307,-0.259147,954.122563,9.999751e-01,-0.016390
3,AP,MCMV,2.168621,29.617424,-27.448803,0.117879,-232.854993,554.694583,0.000000e+00,-14.727043
4,AP,RAP\nMUSIC,2.168621,1.938508,0.230113,0.038230,6.019212,997.935187,5.151930e-08,0.380688
5,AP,eLOR,2.168621,28.174263,-26.005642,0.068478,-379.767978,678.756742,7.571721e-14,-24.018636
6,Champagne,FLEX\nAP,6.459155,2.535125,3.924030,0.129371,30.331603,635.226387,0.000000e+00,1.918339
7,Champagne,FLEX\nMUSIC,6.459155,2.179844,4.279311,0.125823,34.010644,575.894702,3.785861e-14,2.151022
8,Champagne,MCMV,6.459155,29.617424,-23.158269,0.166887,-138.765823,994.983168,2.177147e-13,-8.776321
9,Champagne,RAP\nMUSIC,6.459155,1.938508,4.520647,0.124167,36.407875,548.117871,0.000000e+00,2.302636



 Post-hoc Extended Sources


,A,B,mean(A),mean(B),diff,se,T,df,pval,cohen
0,AP,Champagne,2.150220,6.947038,-4.796818,0.144122,-33.283047,536.787250,0.000000e+00,-2.105005
1,AP,FLEX\nAP,2.150220,3.762971,-1.612751,0.059908,-26.920413,748.635280,2.001732e-13,-1.702596
2,AP,FLEX\nMUSIC,2.150220,3.360309,-1.210089,0.056885,-21.272377,778.488273,3.547163e-13,-1.345383
3,AP,MCMV,2.150220,29.985298,-27.835078,0.115609,-240.769892,558.854964,0.000000e+00,-15.227625
4,AP,RAP\nMUSIC,2.150220,1.748091,0.402129,0.035302,11.390981,952.822155,3.030909e-14,0.720429
5,AP,eLOR,2.150220,28.309607,-26.159387,0.069224,-377.893444,680.347618,4.509726e-13,-23.900080
6,Champagne,FLEX\nAP,6.947038,3.762971,3.184067,0.151137,21.067377,637.370904,4.605205e-13,1.332418
7,Champagne,FLEX\nMUSIC,6.947038,3.360309,3.586729,0.149965,23.917124,620.669074,0.000000e+00,1.512652
8,Champagne,MCMV,6.947038,29.985298,-23.038260,0.180607,-127.560221,949.077787,4.353184e-13,-8.067617
9,Champagne,RAP\nMUSIC,6.947038,1.748091,5.198947,0.143177,36.311226,523.294038,0.000000e+00,2.296524


# Plot brains

In [55]:
import pickle as pkl
import mne
import sys; sys.path.insert(0, '../invert')
from invert.util import pos_from_forward
from copy import deepcopy
import pandas as pd

clim=dict(kind="value", pos_lims=(0.0, 0.01, 1))
pp = dict(surface='inflated', hemi='both', background="white", verbose=0, colorbar=False, time_viewer=False)

sim_type = "extended"
fn = f"evaluation/sim_and_preds_{sim_type}.pkl"
with open(fn, 'rb') as f:
    stc_dict_ext, x_test_ext, y_test_ext, sim_info_ext, _, _ = pkl.load(f)

sim_type = "single"
fn = f"evaluation/sim_and_preds_{sim_type}.pkl"
with open(fn, 'rb') as f:
    stc_dict_sing, x_test_sing, y_test_sing, sim_info_sing, _, _ = pkl.load(f)



# Combine
stc_dict = deepcopy(stc_dict_sing)
for key, value in stc_dict_ext.items():
    stc_dict[key].extend(value)
x_test = np.concatenate([x_test_sing, x_test_ext], axis=0)
y_test = np.concatenate([y_test_sing, y_test_ext], axis=0)
sim_info = pd.concat([sim_info_sing, sim_info_ext])


fwd = mne.read_forward_solution("forward_model/64ch_ico3-fwd.fif", verbose=0)
fwd = mne.convert_forward_solution(fwd, force_fixed=True)
pos = pos_from_forward(fwd)
source_model = fwd['src']
vertices = [source_model[0]['vertno'], source_model[1]['vertno']]

samples = [23, 36, 508, 514]
imgs = []
colorbars = []
names = []
for sample in samples:
    
    tmin = 0
    tstep = 1/1000
    subject = "fsaverage"

    evoked = mne.EvokedArray(x_test[sample].T, info, tmin=0)
    # evoked.plot_joint()
    stc = mne.SourceEstimate(y_test[sample].T, vertices, tmin=tmin, tstep=tstep, 
                            subject=subject, verbose=0)
    first_sample = stc.data[:, 0]
    first_sample /= np.max(abs(first_sample))
    stc.data = np.tile(first_sample, (20,1)).T
    brain = stc.plot(**pp, brain_kwargs=dict(title="True"), clim=clim)
    pp["colorbar"] = True
    brain_cb = stc_list[sample].plot(**pp, brain_kwargs=dict(title=solver), clim=clim)
    pp["colorbar"] = False

    img = brain.screenshot()
    colorbar = brain_cb.screenshot()
    brain.close()
    brain_cb.close()
    imgs.append( img )
    colorbars.append( colorbar )
    names.append("Ground Truth")
    
    for solver, stc_list in stc_dict.items():
        first_sample = stc_list[sample].data[:, 0]
        first_sample /= np.max(abs(first_sample))
        stc_list[sample].data = np.tile(first_sample, (20,1)).T
        if solver == "eLORETA" or solver == "MCMV":
            brain = stc_list[sample].plot(**pp, brain_kwargs=dict(title=solver), clim=dict(kind="value", pos_lims=(0.2, 0.5, 1)))
            pp["colorbar"] = True
            brain_cb = stc_list[sample].plot(**pp, brain_kwargs=dict(title=solver), clim=dict(kind="value", pos_lims=(0.2, 0.5, 1)))
            pp["colorbar"] = False
        else:
            brain = stc_list[sample].plot(**pp, brain_kwargs=dict(title=solver), clim=clim)
            pp["colorbar"] = True
            brain_cb = stc_list[sample].plot(**pp, brain_kwargs=dict(title=solver), clim=clim)
            pp["colorbar"] = False
            
        img = brain.screenshot()
        colorbar = brain_cb.screenshot()
        
        brain.close()
        brain_cb.close()

        imgs.append( img )
        colorbars.append(colorbar)
        names.append(solver)

    No patch info available. The standard source space normals will be employed in the rotation to the local surface coordinates....
    Changing to fixed-orientation forward solution with surface-based source orientations...
    [done]


NameError: name 'stc_list' is not defined